In [56]:
import cv2
import numpy as np
from itertools import count
from skimage.measure import ransac
from skimage.transform import FundamentalMatrixTransform, EssentialMatrixTransform

vid_path = '5.hevc'

def generate_frames(vid_path):
    video = cv2.VideoCapture(vid_path, cv2.CAP_FFMPEG)
    _, prev_frame = video.read()
    for t in count():
      ret, curr_frame = video.read()
      if not ret:
        break
      yield prev_frame, curr_frame
      prev_frame = curr_frame
    video.release()
    cv2.destroyAllWindows()

def extractFeatures(frame):
  orb = cv2.ORB_create()
  # only works on b/w images
  pts = cv2.goodFeaturesToTrack(np.mean(frame,axis=2).astype(np.uint8), 3000, 0.01, minDistance=7)
  # we need kp class to feed it into ORB.compute to get descriptors
  kps = [cv2.KeyPoint(x=f[0][0], y=f[0][1], size=10) for f in pts]
  kps, des = orb.compute(frame, kps)
  return np.array([(kp.pt[0], kp.pt[1]) for kp in kps]), des

def bfmatcher(kps, dess):
  bf = cv2.BFMatcher(cv2.NORM_HAMMING)
  # find closest descriptors between frames, hamming norm for ORB (BRIEF, BRISK...)
  # they are binary string types
  # L1, L2 for SIFT/SURF
  matches = bf.knnMatch(dess[0], dess[1], k=2)
  res = []
  # DMatch obj: distance (the lower the better)
  # trainIdx: index of the descriptor in train desc
  # queryIdx: index of the descriptor in query desc
  # imgIdx: index of the train image
  for m,n in matches:
    # Lowe's ratio test
    # https://stackoverflow.com/questions/51197091/how-does-the-lowes-ratio-test-work
    if m.distance < 0.75 * n.distance:
        kp1 = kps[0][m.queryIdx]
        kp2 = kps[1][m.trainIdx]
        res.append((kp1, kp2))
  res = np.array(res)
  res = normalize(res)
  # prune the outliers by fitting F or E (random sampling of the data)
  assert len(res)>=8, 'not enough points'
  model, inliers = ransac((res[:,0],
                          res[:,1]),
                          FundamentalMatrixTransform, 
                          #EssentialMatrixTransform,
                          min_samples=8,
                          residual_threshold=0.5, 
                          max_trials=200)
  s,v,d = np.linalg.svd(model.params)
  print (v)
  res = denormalize(res)
  #print (f'good kps: {len(res[inliers])/len(res)*100:.2f}% out of {len(res)}')
  return res[inliers, 0], res[inliers, 1]

w = 1164
h = 874
F = 910

K = [[F, 0, w//2],
     [0, F, h//2],
     [0, 0, 1]]

def normalize(x):
  x[:,0] -= w//2
  x[:,1] -= h//2  
  return x
def denormalize(x):
  x[:,0] += w//2
  x[:,1] += h//2
  return x

In [57]:
for p, c in generate_frames(vid_path):

  kps_1, des_1 = extractFeatures(p)
  kps_2, des_2 = extractFeatures(c)

  p1, p2 = bfmatcher([kps_1, kps_2], [des_1, des_2])
  
  for p in p2:
    cv2.circle(c, (int(p[0]), int(p[1])), 1, (255,255,0)) 
  
  for k1, k2 in zip(p1, p2):
    cv2.line(c, tuple(k1.astype(int)), tuple(k2.astype(int)), (0,200,200), 1)
  
  cv2.imshow('v', c)
  key = cv2.waitKey(1)
  if key == ord('q'):
    break
  
cv2.destroyAllWindows()

[5.81032872e-01 2.52534275e-05 6.74124406e-21]
[5.82374639e-01 2.23399441e-05 5.12108683e-19]
[5.15686975e-01 1.97270399e-05 3.82045905e-19]
[5.26122711e-01 1.84746441e-05 2.47505393e-19]
[1.17796851e+00 3.44779838e-05 1.20707060e-18]
[2.46772609e-01 1.47189463e-05 1.56655509e-19]
[7.89454625e-01 2.64327520e-05 3.59303000e-19]
[1.04139573e-01 1.33151909e-05 4.37043306e-19]
[4.50735967e-01 1.81040499e-05 6.10420731e-19]
[1.01626255e+00 3.35950544e-05 3.73190012e-18]
[1.41799993e+00 4.08493869e-05 1.94114369e-19]
[1.16612038e+00 3.61094926e-05 3.16959644e-19]
[1.07222422e+00 3.34297698e-05 1.22066210e-18]
[2.07514285e-01 1.69583938e-05 1.91252547e-19]
[3.05481034e-01 1.54551180e-05 1.47308903e-18]
